In [1]:
import nexradaws
import time
import tempfile
import os
import shutil
import pyart
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature

from cartopy.feature import NaturalEarthFeature
import matplotlib.colors as colors


import numpy as np

import pytz
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
from copy import deepcopy

%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/Users/scollis/miniconda3/envs/chihydro/lib/python3.8/site-packages/xarray/core/pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")
/Users/scollis/miniconda3/envs/chihydro/lib/python3.8/site-packages/xarray/core/pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")
/Users/scollis/miniconda3/envs/chihydro/lib/python3.8/site-packages/xarray/core/pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")
/Users/scollis/miniconda3/envs/chihydro/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/scollis/miniconda3/envs/chihydro/lib/python3.8/site-packages/s

In [2]:
#https://stackoverflow.com/questions/32237862/find-the-closest-date-to-a-given-date/32237949
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))


# This is a VERY LAZY function. 
#No testing and if it gets a file Py-ART can not read it will fail, ungracefully

def get_my_radar(connex, site, this_datetime):
    tlocation = tempfile.mkdtemp()
    these_scans = connex.get_avail_scans(this_datetime.year,this_datetime.month, this_datetime.day, site)
    these_times = [scan.scan_time for scan in these_scans]
    targ = this_datetime
    
    #Need to clean
    these_good_scans = []
    these_good_times = []
    for i in range(len(these_scans)):
        if these_times[i] is not None:
            these_good_times.append(these_times[i])
            these_good_scans.append(these_scans[i])
    
    print(len(these_good_scans), len(these_good_times))

    this_nearest_time = nearest(these_good_times, targ)
    this_index = these_good_times.index(this_nearest_time)
    lcn = templocation
    localfiles = conn.download(these_good_scans[this_index],lcn)
    return pyart.io.read(localfiles.success[0].filepath)

In [3]:
templocation = tempfile.mkdtemp()
conn = nexradaws.NexradAwsInterface()


In [7]:
for i in [16,17,18,19,20,21,22,23]:
    for j in [5,15,25,35,45,55]:
        recent_time_n = datetime(2013, 4, 17, i, j, 0)
        recent_time = recent_time_n.replace(tzinfo=pytz.UTC)
        recent_radar = 'KLOT'
        radar = get_my_radar(conn, recent_radar, recent_time)
        ff = time.strptime(radar.time['units'][14:], '%Y-%m-%dT%H:%M:%SZ')
        strform = time.strftime('%Y%m%d-%H%M%S',ff)
        print(strform)
        base_file = '/Users/scollis/ubrims/out_' + strform
        rain_z = radar.fields['reflectivity']['data'].copy()
        z_lin = 10.0**(radar.fields['reflectivity']['data']/10.)
        rain_z = (z_lin/300.0)**(1./1.4)  #Z=300 R1.4
        radar.add_field_like('reflectivity', 'rain_z',  rain_z, replace_existing = True)
        radar.fields['rain_z']['units'] = 'mm/h'
        radar.fields['rain_z']['standard_name'] = 'rainfall_rate'
        radar.fields['rain_z']['long_name'] = 'rainfall_rate_from_z'
        radar.fields['rain_z']['valid_min'] = 0
        radar.fields['rain_z']['valid_max'] = 500
        pyart.io.write_cfradial(base_file+'-data.nc', radar)



286 286
Downloaded KLOT20130417_160639_V06.gz
1 out of 1 files downloaded...0 errors
20130417-160639
286 286
Downloaded KLOT20130417_161555_V06.gz
1 out of 1 files downloaded...0 errors
20130417-161555
286 286
Downloaded KLOT20130417_162513_V06.gz
1 out of 1 files downloaded...0 errors
20130417-162513
286 286
Downloaded KLOT20130417_163431_V06.gz
1 out of 1 files downloaded...0 errors
20130417-163431
286 286
Downloaded KLOT20130417_164345_V06.gz
1 out of 1 files downloaded...0 errors
20130417-164345
286 286
Downloaded KLOT20130417_165300_V06.gz
1 out of 1 files downloaded...0 errors
20130417-165300
286 286
Downloaded KLOT20130417_170655_V06.gz
1 out of 1 files downloaded...0 errors
20130417-170655
286 286
Downloaded KLOT20130417_171612_V06.gz
1 out of 1 files downloaded...0 errors
20130417-171612
286 286
Downloaded KLOT20130417_172735_V06.gz
1 out of 1 files downloaded...0 errors
20130417-172735
286 286
Downloaded KLOT20130417_173652_V06.gz
1 out of 1 files downloaded...0 errors
201304

In [5]:
 pyart.io.write_cfradial?